# ORCA-PT-1 Hands-On Tutorial
Welcome to the hands-on introduction to the **ORCA-PT-1 Quantum Computer** via the PCSS QAPI.

This notebook includes three practical sections:
- **Time Bin Interferometer (TBI)**: for quantum sampling.
- **Quantum ML Layer (PTLayer)**: for neural network integration.
- **Binary Bosonic Solver (BBS)**: for combinatorial optimization.

Each section includes meaningful use cases to build from.

In [1]:
from pcss_qapi import AuthorizationService
# Log in to PCSS QAPI (interactive prompt will appear)
AuthorizationService.login()


🔐 Authorize Access
----------------------------------------
You are about to be redirected to an authorization server.
There, you will be asked to grant access permissions.
This allows the system to act on your behalf using delegated access.
Please confirm only if you trust this application.
➡️  Click to authorize: https://sso.classroom.pionier.net.pl/auth/realms/Classroom/device?user_code=GNBM-PMMU

✅ Access granted.                                                     


In the next cell we initialize the OrcaProvider, which can list available quantum simulators and real devices. It helps to check the computing backends accessible after login.

In [2]:
from pcss_qapi.orca.provider import OrcaProvider
provider = OrcaProvider() 

print("Available simulators:", provider.available_backends(simulators=True))
print("Available real devices:", provider.available_backends(simulators=False))

Available simulators: ['single_loop_simulator', 'multi_loop_simulator']
Available real devices: ['ORCA-PT-1-A', 'ORCA-PT-1-B']


## 1. Time Bin Interferometer (TBI)

The Time Bin Interferometer (TBI) is used to simulate quantum interference from multi-photon inputs over looped time-bin paths.  
**Sampling** refers to drawing repeated outcomes from a quantum circuit probabilistically, akin to sampling from a quantum distribution.

In [ ]:
# Choose backend
backend = provider.get_backend("multi_loop_simulator")

# Get the tbi from the chosen backend
tbi = backend.get_tbi()
tbi.draw(input_state = [1, 0, 1, 0])

# Tbi's sample method runs sampling
samples = tbi.sample(input_state=[1, 0, 1, 0], theta_list=[0.2, 1.0, 0.2], n_samples=50)

print("TBI samples:", samples)

Observe how output changes based on the changes of theta list. If we set the input_state to [1, 0] and leave default value of loop lengths, which is [1], there will be only one beam splitter. There is a simple way to calculate number of beam splitters in the tbi: Just substract length of the loop from length of the input state for every loop and sum over all loops. Alternatively you can use ptseries' built in function calculate_parameters (It requires importing from the right file).

In [ ]:
tbi.draw(input_state = [1,0])

In [6]:
import numpy as np

samples = tbi.sample(input_state=[1, 0], theta_list=[0], n_samples=50)

print("TBI samples:", samples)

TBI samples: {(1, 0): 50}


In [ ]:
samples = tbi.sample(input_state=[1, 0], theta_list=[np.pi / 2], n_samples=50)

print("TBI samples:", samples)

Even though the input state stays the same, change in the angle of beam splitter causes photons to end up in the other detector. Notice that angles are passed as radians, so beam splitter with angle set to integer * Pi will work as it would have it's angle set to 0. You can also change the input_state however you want, but keep in mind, that bigger input state complexifies the circuit and lenghtens processing time. It is worth to mention that on real hardware length of the input_state doesn't have that much impact on the processing time as number of photons does. There can be a situation where circuit with input_state [1, 0, 0, 0, 0] will be much faster than [1, 1, 1].

There are two more parameters with which you can directly influence the output of the sample method. First one is really usefull - n_tiling. Try figuring out what it does.  
  
Tip: Make sure that length of theta list is multiplied by the number you chose n_tiling to be.

In [ ]:
samples = tbi.sample(input_state=[1, 0], theta_list=[np.pi / 3, np.pi / 2], n_samples=50, n_tiling = 2)

print("TBI samples:", samples)

Tiling is an alternative way of calculating too big circuits. If you use tiling, n_tiling circuits are run separately and outputs are classically merged into one output. Tiling scales linearly, which can cause a situation in which running many smaller circuits is faster than running one big. In this case you'd lose on quantum (because photons don't interact between tiles), but you gain time.

The other parameter is there purely for convenience. It's caleed output_format and thanks to it you can choose the output to be in a dictionary, tuple, list or array.

In [ ]:
samples = tbi.sample(input_state=[1, 0], theta_list=[0], n_samples=50, output_format="dict") # "tuple", "list", "array"

print("TBI samples:", samples)

You can also influence the output indirectly through TBI parameters. Most of them are there to make simulations not perfect (and you probably will never use them, unless you are doing some deep reserach), but there are two that can be used quite often: n_loops and loop_lengths. See for yourself how they affect the output and structure of the TBI.


In [ ]:
tbi = backend.get_tbi(n_loops = 3, loop_lengths=[1, 2, 3])

tbi.draw(input_state = [1, 0, 1, 0])

samples = tbi.sample(input_state=[1, 0, 1, 0], theta_list=[0.1, 0.4, 0.2, 1, 1.5, 2], n_samples=50)

print("TBI samples:", samples)

N_loops is necessary only when you just want to specify the number of loops and want to use lengths of 1. If you specify loop_lengths, n_loops is not necessary. But what does the length of the loop mean? For example: If you set loop length to 2, only qumodes that have one qumode between them will be connected with beam splitter. If set to 3, then qumodes with 2 qumodes between them will be connected.

## Universal Parameters

- **`n_loops`** *(int)*  
  Number of loops in the TBI.  

- **`loop_lengths`** *(list of int)*  
  List of lengths of the loops.  


## Simulator Parameters

- **`distinguishable`** *(bool)*  
  If `True`, photons behave like non-quantum (classical) particles.  

- **`bs_loss`** *(float, [0–1])*  
  Probability of photon loss on every beam splitter.  

- **`bs_noise`** *(float, [0-1])*  
  Percentage by which the beam splitter angle may vary.  

- **`input_loss`** *(float, [0–1])*  
  Probability of photon loss at the input.  

- **`detector_efficiency`** *(float, [0–1])*  
  Probability of correctly detecting a photon.  

- **`n_signal_detectors`** *(int)*  
  Number of detectors in the mode.  

- **`g2`** *(float)*  
  Autocorrelation of a pair of generated photons.  

- **`afterpulse_probability`** *(float, [0–1])*  
  Probability of another, unwanted photon being generated.  


## Real Hardware Parameters

- **`postselection`** *(bool)*  
  If `True`, enables postselection that reduces errors.  

- **`postselection_threshold`** *(int)*  
  The threshold for postselection. Defaults to None. If None, and postselection is True, then the threshold is set to the number of input photons.  


## Playground  
Try experimenting with other parameters. In order to use them you have to pass them to the get_tbi method as a dictionary.

In [8]:
provider.get_task_ids()

['6d37e86d-7b1e-4c84-a279-bb2be4203b13',
 '20885320-a9a9-4af2-bc35-cf290b7f9b7b']

In [ ]:
simulator_parameters = {
    "distinguishable": True,
}

tbi = backend.get_tbi(simulator_params=simulator_parameters)

samples = tbi.sample(input_state=[1, 0, 1, 0], theta_list=[1, 0.5, 2], n_samples=50)

print("Tbi samples", samples)

## Here you can use the real quantum computer - PT-1  
But remember that there are only so many available ones, so if you do not have to, do not run thousands of samples. Real hardware is rather small. It has 8 qumodes and 2 loops which means that the highest length of the input state can be 8 and you can use either 1 or 2 loops, but the lengths are fixed to 1. Also keep in mind that producing singular photons is really hard, so running circuits with more than 4 photons can already be slow. Producing these photons requires shooting laser into a crystal, but in order for that to work the source of the laser has to warm up. So if you are the first user to use the PT-1 it may take additional time. It all sounds like PT-1 has many limitations, but it is the first of it's kind and it's next generation is already being tested and the results are quite convincing. By researching PT-1 we pave the way for the future.

In [7]:
backend = provider.get_backend("ORCA-PT-1-B")

tbi = backend.get_tbi()

samples = tbi.sample(input_state=[1, 0], theta_list=[np.pi/4], n_samples=10)

print("TBI samples:", samples)

TBI samples: {(0, 1): 3, (1, 0): 7}


## 2. Quantum Machine Learning with PTLayer
You can integrate ORCA-PT-1 as a trainable quantum layer inside classical PyTorch models.

In [ ]:
# Go back to the simulator if you used real hardware

backend = provider.get_backend("multi_loop_simulator")

In [ ]:
import torch

# Simulate 3 input features representing encoded quantum parameters
ptlayer = backend.get_ptlayer(in_features=3)

x = torch.tensor([[1.0, -0.5, 0.3]], dtype=torch.float32)
output = ptlayer(x)
print("Output:", output)

### Example Use Case: Classifier Feature Injection
You can place this layer in the middle of a classical network to enhance non-linearity with quantum interaction.

In [ ]:
# Example: feed-forward classifier with quantum middle layer
class HybridNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(4, 3)
        self.q = backend.get_ptlayer(in_features=3)
        self.fc2 = torch.nn.Linear(4, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.q(x)
        return self.fc2(x)

model = HybridNet()
example_input = torch.randn((1, 4))
print("Predicted output:", model(example_input))

### Multi-Class Classification with Iris Dataset and PTLayer

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA 

iris = load_iris()
X = iris.data
y = iris.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)

train_ds = TensorDataset(X_train_t, y_train_t)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)


In the next cell we apply Principal Component Analysis (PCA) to reduce the Iris dataset to 2D and plot the classes, allowing for easy visualization of class separation.

In [ ]:
# Visualize Iris dataset after PCA
pca = PCA(n_components=2)
X_2d = pca.fit_transform(X)
plt.figure(figsize=(6, 4))
for cls in range(3):
    plt.scatter(X_2d[y == cls, 0], X_2d[y == cls, 1], label=iris.target_names[cls])
plt.title('Iris Dataset After PCA')
plt.legend()
plt.show()

Next, let's create a simple PyTorch hybrid classifier (placeholder for a quantum-classical hybrid). The network stacks fully-connected layers and outputs class probabilities with softmax for the Iris dataset.

In [ ]:
# Define Quantum-Classical Hybrid Network
class HybridNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc1 = torch.nn.Linear(4, 8)
        self.fc2 = torch.nn.Linear(8, 3)
        self.q_layer = backend.get_ptlayer(in_features=3)
        self.fc3 = torch.nn.Linear(4, 3)  # output 3 classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.q_layer(x)
        x = self.fc3(x)
        return torch.softmax(x, dim=1)

Now we will train the model. We will apply Adam optimizer and run 50 epochs. Each 5th epoch we will print the loss for the epoch.

In [ ]:
# Train Model
model = HybridNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
import torch.nn.functional as F
losses = []

for epoch in range(40):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = F.cross_entropy(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss / len(train_loader))
    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Loss: {losses[-1]:.4f}")

Now let's see how the loss decreases though the training and what is the final accuracy.

In [ ]:
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True)
plt.show()

In [ ]:
model.eval()
with torch.no_grad():
    preds = model(X_test_t)
    acc = (preds.argmax(1) == y_test_t).float().mean()
print(f"Test Accuracy: {acc:.2f}")

Now let's see how many samples were classified correctly and not.

In [ ]:
# Ensure model is in eval mode and gradients are off 
model.eval()
with torch.no_grad():
    preds = model(X_test_t)
    predicted_labels = preds.argmax(1)
    incorrect = (predicted_labels != y_test_t)

plt.figure(figsize=(6, 4))
    
for cls in range(3):
    plt.scatter(X_test[y_test == cls, 0], X_test[y_test == cls, 1], label = iris.target_names[cls])
    
plt.scatter(X_test[incorrect, 0], X_test[incorrect, 1], c='red', marker='x')

# Visualization
plt.title('Iris Dataset After PCA with errors marked')
plt.legend()
plt.show()


PTLayer as well as TBI also has some additional parameters.

## Layer Parameters

- **`input_state`** *(list of int)*  
  State that will be passed to the TBI.  

- **`in_features`** *(list of float)*  
  Angles of the beam splitters, determined by the input of the layer.  
  If set to a list with samller length than the number of beam splitters, the remaining beam splitters become trainable parameters.  

- **`tbi_params`** *(dict)*  
  Parameters of the TBI to be used in the `PTLayer`.  

- **`observable`** *(str)*  
  Method of interpreting detected photons:  
  - `"mean"` → use `Mean()` (single-mode mean photon numbers).  
  - `"correlations"` → use `Correlations()` (two-point photon correlators).  
  - `"covariances"` → use `Covariances()` (two-point photon covariances).  
  - `"single-sample"` → use `SingleSample()` (one Monte Carlo sample per forward pass).  

- **`n_samples`** *(int)*  
  Number of samples to draw.  

- **`n_tiling`** *(int)*  
  Number of tiles. Tiles are replicas of the circuit that are later postprocessed to behave as one big circuit.  

- **`gradient_mode`** *(str)*  
  Method of gradient calculation:  
  - `"parameter-shift"`  
  - `"finite-difference"`  
  - `"spsa"`  

- **`gradient_delta`** *(float)*  
  Value used for gradient calculation.  


## Playground
Check out how other parameters influence the computation.

## 3. Binary Bosonic Solver (BBS)

The BBS algorithm solves **combinatorial optimization problems** using quantum-classical loop. Certain output state is obtained from TBI and mapped to potential binary solution of a problem. After that bitflip model is applied negating certain values of the solution based on probabilities. These probabilities are trainable parameters of the model. After negating the bits, the solution is evaluated based on an objective. Based on the value of evaluation score, known as cost or energy, model parameters are updated using gradient methods.

Here you will solve an instance of the Max-Cut problem.  

The Max-Cut problem is about dividing the set of vertices of a graph into two groups so that the number of edges connecting these groups is as large as possible. In other words, we want to “cut” the graph into two parts and count how many edges cross between them — and the goal is to maximize that number.

In [ ]:
import networkx as nx

# Define a small graph.

edges = [(0, 1), (1, 2), (2,3), (3,0)]

G = nx.Graph()
G.add_edges_from(edges)
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, node_size=800, font_color="white")
plt.show()

def objective_function(x):
    return sum([int(x[i] == x[j]) for i, j in edges])

BBS always tries to minimize given objective function, so make sure that objective function does what you want and not the opposite. Here we want to maximize the number of cuts, but since BBS minimizes we have to negate the objective.

In [ ]:
bbs = backend.get_bbs(pb_dim = 4, objective = objective_function)
bbs.solve(updates=20, print_frequency=2)

best_energy = bbs.best_cost
solution = bbs.best_solution

print("Best energy:", best_energy)

colors = ["red" if solution[i] == 0 else "blue" for i in G.nodes()]

nx.draw(G, pos, with_labels=True, node_color=colors, node_size=800, font_color="white")
plt.show()

## BBS Parameters

- **`pb_dim`** *(int)*  
  Number of decision variables.  

- **`objective`** *(array or callable)*  
  QUBO matrix or non-QUBO function.  

- **`input_state`** *(list of int)*  
  State that will be passed to the TBI (constant).  

- **`tbi_params`** *(dict)*  
  Parameters of the TBI used for sampling.  

- **`n_samples`** *(int)*  
  Number of samples.  

- **`gradient_delta`** *(float)*  
  Value used for gradient calculation.  

- **`gradient_mode`** *(str)*  
  Way of calculating the gradient.  

- **`spsa_params`** *(dict)*  
  Parameters for SPSA calculation.  

- **`sampling_factor`** *(int)*  
  Number of times quantum samples are passed through the classical flipping layer.  

- **`entropy_penalty`** *(float)*  
  Factor that incentivises convergence of the bit-flip model.  


## Playground
Check out how other parameters influence the computation.